In [ ]:
print("reddit")

In [ ]:
pip install praw python-dotenv

In [ ]:
import praw
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Retrieve credentials from environment variables
client_id = os.getenv("REDDIT_CLIENT_ID")
client_secret = os.getenv("REDDIT_CLIENT_SECRET")
user_agent = os.getenv("REDDIT_USER_AGENT")

# Reddit API credentials
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

# Subreddit name
subreddit_name = "cosmosnetwork"  # Replace with the desired subreddit name
subreddit = reddit.subreddit(subreddit_name)

# Fetch the latest 10 posts
posts = []
for submission in subreddit.new(limit=10):  # Use .new() for the newest posts
    posts.append(submission.title + ": " + submission.selftext)

# Print the fetched posts
print("Fetched posts from subreddit:")
for post in posts:
    print(post)

In [ ]:
pip install transformers torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Authenticate Hugging Face (required for gated models)
# Public models do not need this step if you skip the `use_auth_token` parameter.

# Load a summarization model from Hugging Face
# Replace with "meta-llama/Llama-2-7b-hf" if you have access or use a public model like t5-small.
model_name = "t5-small"  # Change this to a different model if needed
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Using the posts fetched from Cell-1 (from the `posts` list)
input_text = "Summarize the following posts into a single tweet:\n" + "\n".join(posts)

# Tokenize input text
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

# Generate summary
summary_ids = model.generate(
    inputs.input_ids,
    max_length=200,           # Maximum length of the summary
    num_beams=5,             # Beam search for better quality
    early_stopping=True      # Stop early when the result is stable
)

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("\nSummary:")
print(summary)

In [ ]:

import praw
import os
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load environment variables from the .env file
load_dotenv()

# Retrieve credentials from environment variables
client_id = os.getenv("REDDIT_CLIENT_ID")
client_secret = os.getenv("REDDIT_CLIENT_SECRET")
user_agent = os.getenv("REDDIT_USER_AGENT")
redirect_uri=os.getenv("REDDIT_REDIRECT_URI")

# Reddit API credentials
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent,
    redirect_uri=redirect_uri    
)

# Step 1: OAuth2 Login
print("Please visit this URL to authorize the app: ", reddit.auth.url(['identity', 'submit'], 'unique_state', 'permanent'))

# After visiting the URL, the user will receive a code. Enter it here.
code = input("Enter the code you received: ")

# Step 2: Use the code to get the access token
reddit.auth.authorize(code)


# Now we are logged in and can post to the subreddit.

# Subreddit name
subreddit_name = "cosmosnetwork"  # Replace with the desired subreddit name
subreddit = reddit.subreddit(subreddit_name)

# Example posts fetched from Reddit (Replace this with the actual posts)
posts = [
    "Post 1: Reddit's blockchain feature is launching soon.",
    "Post 2: New updates from Avail's latest partnerships.",
    "Post 3: Insights into modular blockchain development."
]  # Replace this with your fetched posts

# Summarizing the posts
model_name = "t5-small"  # Change this to a different model if needed
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Prepare text for summarization
input_text = "Summarize the following posts into a single tweet:\n" + "\n".join(posts)

# Tokenize input text
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

# Generate summary
summary_ids = model.generate(
    inputs.input_ids,
    max_length=50,           # Maximum length of the summary
    num_beams=5,             # Beam search for better quality
    early_stopping=True      # Stop early when the result is stable
)

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("\nSummary:")
print(summary)

# Step 3: Post the summary to Reddit
title = "Summary of the latest posts in the Avail subreddit"
subreddit.submit(title, selftext=summary)  # Submit the summary as a post

print(f"Summary successfully posted to {subreddit_name}!")